In [118]:
import sqlite3
import pandas as pd
faili_nimi = 'run_01_gpt-35-turbo.db'


In [119]:
# ühendame algsed andmed lisa prompti andmetega
'''
conn1 = sqlite3.connect('run_01_gpt-35-turbo.db')  # main db
conn2 = sqlite3.connect('run_01_gpt-35-turbo_a.db')  # secondary db

conn1.execute("ATTACH DATABASE 'run_01_gpt-35-turbo_a.db' AS db2;")

# Replace 'my_table' with your actual table name
conn1.execute("""
    INSERT INTO model_annotations
    SELECT * FROM db2.model_annotations;
""")

conn1.commit()
conn1.close()
conn2.close() 
'''

'\nconn1 = sqlite3.connect(\'run_01_gpt-35-turbo.db\')  # main db\nconn2 = sqlite3.connect(\'run_01_gpt-35-turbo_a.db\')  # secondary db\n\nconn1.execute("ATTACH DATABASE \'run_01_gpt-35-turbo_a.db\' AS db2;")\n\n# Replace \'my_table\' with your actual table name\nconn1.execute("""\n    INSERT INTO model_annotations\n    SELECT * FROM db2.model_annotations;\n""")\n\nconn1.commit()\nconn1.close()\nconn2.close() \n'

In [120]:
# vaatame andmeid
conn = sqlite3.connect(faili_nimi)

df = pd.read_sql_query("SELECT * FROM model_annotations;", conn)
conn.close()
df

,input,model,semantic_class,response
0,Nõugaste\nturbaauk\nlillepaviljon\ntranspordik...,gpt-35-turbo,nonsense_words,"{\n ""Nõugaste"": 0,\n ""turbaauk"": 1,\n ..."
1,aedik\nsekretär\ntoosama\nveevärk\nranne\nkiir...,gpt-35-turbo,nonsense_words,"```json\n{""aedik"":0}\n{""sekretär"":1}\n{""toosam..."
2,keep\nLembit\nrevüüteater\nfonoteek\nVarasem\n...,gpt-35-turbo,nonsense_words,"{\n ""keep"": ""1"",\n ""Lembit"": ""1"",\n ""..."
3,advokaadibüroo\nmasšuud\ntöökäsi\npuiatu\nSopo...,gpt-35-turbo,nonsense_words,"```json\n{""advokaadibüroo"":1}\n{""masšuud"":0}\n..."
4,potilill\nshamaanipaar\naastapäevanäitus\nkait...,gpt-35-turbo,nonsense_words,"```json\n{\n ""potilill"": 1,\n ""shamaanip..."
...,...,...,...,...
709,kombinesoon\nlastehommik\nnelipühikirik\nHiire...,gpt-35-turbo,spatial_region,"```json\n{\n ""kombinesoon"": 0,\n ""lasteh..."
710,pronksikoht\npöördtool\nMacau\ntagavedru\nradi...,gpt-35-turbo,spatial_region,"```json\n{""pronksikoht"":0}\n{""pöördtool"":0}\n{..."
711,Gilde\nbüst\npadi\npensioniskeem\nkuuldemäng\n...,gpt-35-turbo,spatial_region,"{\n\t""Gilde"": 0,\n\t""büst"": 0,\n\t""padi"": 0,\n..."
712,juunis-juuli\nlaskmistrenn\ngraafik\nsõidurada...,gpt-35-turbo,spatial_region,"{\n ""juunis-juuli"": 0,\n ""laskmistrenn"":..."


In [121]:
df['response'][0]

'{\n    "Nõugaste": 0,\n    "turbaauk": 1,\n    "lillepaviljon": 1,\n    "transpordikomisjon": 1,\n    "turismihooaeg": 1,\n    "tsiviilvorm": 1,\n    "villipubi": 1,\n    "Svalbard": 0,\n    "proua": 1,\n    "rumm": 1\n}'

In [122]:
import pandas as pd
import re

# leiame 'response' veerust kõik sõnade ja väärtuste paarid
def extract_pairs(text):
    if not isinstance(text, str):
        return []
    
    matches = re.findall(r'"([^"]+)"\s*:\s*"?(0|1)"?', text) #leiab järgmisel kujul sõned ["sõna": 0, "sõna": 1,"sõna": "0","sõna": "1"]
    return [(word, int(value)) for word, value in matches]


In [123]:
# loome uue dataframe'i, kus on iga leitud sõna kohta rida
# igas reas on sõna, semantiline_klass ja ennustus

rows = []

# Loop through each row in the original DataFrame
for _, row in df.iterrows():
    semantic_class = row['semantic_class']
    word_value_pairs = extract_pairs(row['response'])
    
    for lemma, value in word_value_pairs:
        rows.append({
            'lemma': lemma,
            'väärtus': int(value),
            'semantiline_klass': semantic_class
        })

simple_df = pd.DataFrame(rows)
simple_df

,lemma,väärtus,semantiline_klass
0,Nõugaste,0,nonsense_words
1,turbaauk,1,nonsense_words
2,lillepaviljon,1,nonsense_words
3,transpordikomisjon,1,nonsense_words
4,turismihooaeg,1,nonsense_words
...,...,...,...
6729,ovaalsaal,0,spatial_region
6730,softivahetus,0,spatial_region
6731,Elbrus,1,spatial_region
6732,dj,0,spatial_region


In [124]:
# Saame loodud dataframe'ist lõpliku tulemuse, kus on semantilised_klassid veergude nimed
final_df = simple_df.pivot_table(index='lemma', 
                            columns='semantiline_klass', 
                            values='väärtus', 
                            aggfunc='first') 




final_df.columns.name = None 
final_df = final_df.reset_index()
final_df

,lemma,belongings,clothing_items,edibles,enclosures_and_vehicles,geographical_locations,institutions,nonsense_words,organisations_and_events,persons,platforms_and_floors,spatial_directions,spatial_region,time_expressions,vegetative_enclosure
0,3Dmark01SE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,60C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7000rpm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A-kategooria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Acapulco,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,õppelaenupirukas,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
484,õppepolk,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
485,ühistransport,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
486,üleviimine,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [125]:
# salvestame loodud dataframe'i
final_df.to_csv('GPT3_5_tulemused.csv', index=False)
#final_df.to_csv('GPT4_0_tulemused.csv', index=False)